In [ ]:
import numpy as np
import ising_model

%load_ext autoreload
%autoreload 2
%load_ext Cython
%load_ext line_profiler

lattice_size = 1024
bond_energy = 1
lattice2 = np.random.choice([-1, 1], lattice_size**2).reshape(lattice_size, lattice_size)

# ising = ising_model.IsingModel(lattice_size, 1, 4, "hi", 100)

In [ ]:
# OpenMP fails when you don't use gcc-5 or another OpenMP-enabled compiler.
%%cython --annotate --compile-args=-fopenmp --link-args=-fopenmp --force
import numpy as np
import cython
import cython.parallel
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def calculate_lattice_energy(np.ndarray[np.int_t, ndim=2] lattice, int lattice_size, int bond_energy):
    """Calculate the energy of the lattice using the Ising model Hamiltonian in zero-field."""
    cdef int energy = 0
    cdef int y, x, center, offset_y, offset_x, xnn, ynn
    with nogil, cython.parallel.parallel(num_threads=4):
        for y in cython.parallel.prange(lattice_size):
            offset_y = y + 1
            if y + 1 >= lattice_size:
                offset_y = offset_y - lattice_size
            for x in range(lattice_size):
                offset_x = x + 1
                if x + 1 >= lattice_size:
                    offset_x = offset_x - lattice_size
                center = lattice[y, x]
                xnn = lattice[y, offset_x]
                ynn = lattice[offset_y, x]
                if xnn == center:
                    energy += -1 * bond_energy
                else:
                    energy += bond_energy
                if ynn == center:
                    energy += -1 * bond_energy
                else:
                    energy += bond_energy
    return energy

In [ ]:
%timeit calculate_lattice_energy(lattice2, lattice_size, bond_energy)

In [ ]:
%timeit ising.calculate_lattice_energy()

# 1024 by 1024
- Plain function: 1 loop, best of 3: 669 ms per loop
- Class method: 1 loop, best of 3: 760 ms per loop
- Simple Cython: 1 loop, best of 3: 500 ms per loop
- Some typing: 1 loop, best of 3: 420 ms per loop
- Full cython solution: 100 loops, best of 3: 3.57 ms per loop
- Including parallel execution: 100 loops, best of 3: 1.64 ms per loop
Both plain and cython versions give same output, 395 times speedup

In [ ]:
print(calculate_lattice_energy(lattice2, lattice_size, bond_energy))

In [ ]:
print(ising.calculate_lattice_energy())

In [ ]:
def lattice_energy(lattice, lattice_size, bond_energy):
    """Calculate the energy of the lattice using the Ising model Hamiltonian in zero-field."""
    energy = 0
    for y in range(lattice_size):
        offset_y = (y + 1) % lattice_size
        current_row = lattice[y]
        next_row = lattice[offset_y]
        for x in range(lattice_size):
            center = current_row[x]
            offset_x = (x + 1) % lattice_size
            if current_row[offset_x] == center:
                energy -= bond_energy
            else:
                energy += bond_energy
            if next_row[x] == center:
                energy -= bond_energy
            else:
                energy += bond_energy
    return energy

In [ ]:
print(lattice_energy(lattice2, lattice_size, bond_energy))

In [ ]:
import cy_ising_model as cy

In [ ]:
%timeit cy.calculate_lattice_energy(lattice2, lattice_size, bond_energy)